In [18]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [19]:
with open('Dataset(intents).json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [20]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [21]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [22]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 12)                204       
                                                                 
Total params: 16,748
Trainable params: 16,748
Non-trainable params: 0
__________________________________________________

In [23]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 12ms/step - loss: 2.4847 - accuracy: 0.0652
Epoch 2/500
2/2 [==============================] - 0s 14ms/step - loss: 2.4829 - accuracy: 0.0652
Epoch 3/500
2/2 [==============================] - 0s 13ms/step - loss: 2.4814 - accuracy: 0.0652
Epoch 4/500
2/2 [==============================] - 0s 14ms/step - loss: 2.4802 - accuracy: 0.0652
Epoch 5/500
2/2 [==============================] - 0s 10ms/step - loss: 2.4792 - accuracy: 0.0652
Epoch 6/500
2/2 [==============================] - 0s 14ms/step - loss: 2.4781 - accuracy: 0.0652
Epoch 7/500
2/2 [==============================] - 0s 12ms/step - loss: 2.4770 - accuracy: 0.0652
Epoch 8/500
2/2 [==============================] - 0s 11ms/step - loss: 2.4761 - accuracy: 0.0652
Epoch 9/500
2/2 [==============================] - 0s 14ms/step - loss: 2.4752 - accuracy: 0.0652
Epoch 10/500
2/2 [==============================] - 0s 11ms/step - loss: 2.4744 - accuracy: 0.1304
Epoch 11/500
2/2 [=

In [24]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
pip install colorama

In [27]:



import colorama
colorama.init()
from colorama import Fore, Style, Back
import random
import pickle

with open("Dataset(intents).json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hello
1/1 [==============================] - 0s 77ms/step
ChatBot: Hello
User: who are you
1/1 [==============================] - 0s 30ms/step
ChatBot: I'm charlie, an Artificial Intelligent bot
User: need help
1/1 [==============================] - 0s 35ms/step
ChatBot: Tell me your problem to assist you
User: create a new account
1/1 [==============================] - 0s 41ms/step
ChatBot: Just go to our web site and follow the guidelines to create a new account
User: thanks
1/1 [==============================] - 0s 33ms/step
ChatBot: Happy to help!
User: ask me a riddle
1/1 [==============================] - 0s 39ms/step
ChatBot:  How can a girl go 25 days without sleep?.....She sleeps and night!
User: riddle
1/1 [==============================] - 0s 38ms/step
ChatBot: What has a thumb and four fingers but is not actually alive?.....Your Gloves!
User: who inspires you
1/1 [==============================] - 0s 55ms/step
ChatBot:

Start messaging with the bot (type quit to stop)!
